In [5]:
import mysql.connector
from mysql.connector import Error
import pandas as pd


#### CONNECT TO GOOGLE CLOUD MYSQL
config = {
    'user': 'root',
    'password': 'txybkx5kPMpko8C6',
    'host': '35.238.14.103', 
    'database': 'ddx_db'
}
cnxn = mysql.connector.connect(**config)

#### BASIC QUERY FUNCTIONS

# Read query, something like a select
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")
        
### HELPER FUNCTIONS

# Input: list from query, col names
def list_to_df(ls, cols):
    return pd.DataFrame(ls, columns=cols)



In [2]:
#Query NBA Game Data to get dataset


q1 = """
SELECT * FROM nba_game_data;
"""

nba_game_data = read_query(cnxn, q1)
#print(nba_game_data)
cols = box_stats = ['id', 'gid', 'team', 'fg', 'fga', 'fg_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'off_rtg', 'def_rtg', 'pts', 'date']
ngd_df = pd.DataFrame(nba_game_data, columns=cols)

print(ngd_df.columns)
ngd_df.head()

Index(['id', 'gid', 'team', 'fg', 'fga', 'fg_pct', 'fg3', 'fg3a', 'fg3_pct',
       'ft', 'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov',
       'pf', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct',
       'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct',
       'tov_pct', 'off_rtg', 'def_rtg', 'pts', 'date'],
      dtype='object')


,id,gid,team,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,...,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,off_rtg,def_rtg,pts,date
0,1,/boxscores/202012230TOR.html,nop,42,80,0.525,19,42,0.452,10,...,22.2,84.1,56.3,71.4,12.5,16.3,21.3,108.6,95.2,20201223
1,2,/boxscores/202012230TOR.html,tor,38,89,0.427,14,46,0.304,9,...,15.9,77.8,43.8,68.4,9.6,13.2,17.5,95.2,108.6,20201223
2,3,/boxscores/202012310TOR.html,nyk,32,88,0.364,3,36,0.083,16,...,17.0,81.3,47.5,65.6,6.0,9.7,13.4,83.7,100.8,20201231
3,4,/boxscores/202012310TOR.html,tor,34,83,0.410,17,52,0.327,15,...,18.8,83.0,52.5,61.8,7.1,11.5,14.8,100.8,83.7,20201231
4,5,/boxscores/202101040TOR.html,bos,41,84,0.488,15,29,0.517,29,...,29.3,84.6,60.2,46.3,2.9,17.0,16.1,120.3,108.9,20210104


In [46]:
## HELPER QUERY FUNCTIONS

# Return df of merged game data for opponent stats
# Does not have most up to date betting line data
def merged_nba_game_data_df():
    # Query NBA Game Data to get dataset
    q1 = """
    SELECT * FROM nba_game_data;
    """

    nba_game_data = read_query(cnxn, q1)
    #print(nba_game_data)
    cols = box_stats = ['id', 'gid', 'team', 'fg', 'fga', 'fg_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'off_rtg', 'def_rtg', 'date']
    ngd_df = pd.DataFrame(nba_game_data, columns=cols)


    #print(ngd_df.team)

    df_full_game_join = ngd_df.join(ngd_df.shift(-1).add_prefix('away_'))
    df_full_game_join[1::2] = ''
    df_full_game_join = df_full_game_join[df_full_game_join.gid != '']
    cols1 = ['team', 'away_team', 'pts', 'away_pts']
    df_full_game_join['key'] = df_full_game_join[cols1].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
    df_full_game_join['key'] = df_full_game_join['key'].map(lambda x: str(x)[:-2])
    return df_full_game_join

# Query Betting line data and join with all opponent data
def merged_nba_gd_bl_df():
    df_full_game_join = merged_nba_game_data_df()

    #print(df_full_game_join.key)


    # Query or use csv of line data
    q2 = """
    SELECT * FROM betting_lines;
    """

    betting_lines = read_query(cnxn, q2)
    #print(betting_lines)
    bl_df = pd.DataFrame(betting_lines, columns=['id', 'h_team', 'a_team', 'open_spread', 'close_spread', 'open_total', 'close_total', 'home_ml', 'away_ml', 'home_score', 'away_score', 'date'])
    #print(bl_df.head(50))


    # APPARENTLY THE DF IS REVERESED
    cols = ['a_team', 'h_team', 'away_score', 'home_score']
    bl_df['key'] = bl_df[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
    print(bl_df.key)
    merge_df = bl_df.merge(df_full_game_join, how='left', on='key')
    return merge_df

# Create average of all df for all teams
def avg_nba_game_data_df():
    df = merged_nba_game_data_df()
    stat_cols = cols = ['fg', 'fga', 'fg_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft', 'fta',
       'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts',
       'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'orb_pct',
       'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct',
       'off_rtg', 'def_rtg', 'away_fg', 'away_fga', 'away_fg_pct', 'away_fg3',
       'away_fg3a', 'away_fg3_pct', 'away_ft', 'away_fta', 'away_ft_pct',
       'away_orb', 'away_drb', 'away_trb', 'away_ast', 'away_stl', 'away_blk',
       'away_tov', 'away_pf', 'away_pts', 'away_ts_pct', 'away_efg_pct',
       'away_fg3a_per_fga_pct', 'away_fta_per_fga_pct', 'away_orb_pct',
       'away_drb_pct', 'away_trb_pct', 'away_ast_pct', 'away_stl_pct',
       'away_blk_pct', 'away_tov_pct', 'away_off_rtg', 'away_def_rtg']
    
    df = df.drop(columns=['id', 'gid', 'date', 'key', 'away_team', 'away_date', 'away_id', 'away_gid'])
    df[stat_cols] = df[stat_cols].apply(pd.to_numeric)
    df = df.groupby('team', axis=0).mean().reset_index() 
    return df
    


In [47]:
avg_nba_game_data_df()

   team         fg         fga    fg_pct        fg3       fg3a   fg3_pct  \
0   atl  40.000000   88.200000  0.454600  12.700000  37.000000  0.336400   
1   bos  42.000000   87.600000  0.481400  11.700000  29.500000  0.398700   
2   brk  45.222222   89.888889  0.502667  15.555556  38.222222  0.413000   
3   chi  44.300000   89.000000  0.498800  13.600000  35.200000  0.379300   
4   cho  42.150000   93.125000  0.452900  19.225000  40.125000  0.477000   
5   cle  36.800000   86.100000  0.426900   8.800000  26.700000  0.327000   
6   dal  40.846154   88.846154  0.459231  12.230769  37.153846  0.328462   
7   den  43.900000   88.400000  0.498400  13.900000  34.900000  0.394900   
8   det  40.333333   90.555556  0.444889  12.444444  37.111111  0.334444   
9   gsw  39.125000   92.250000  0.429000  12.500000  37.625000  0.335375   
10  hou  47.205128   98.589744  0.478359  24.794872  48.538462  0.500846   
11  ind  43.875000   90.875000  0.482375  12.125000  35.125000  0.340625   
12  lac  40.

,team,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,...,away_fta_per_fga_pct,away_orb_pct,away_drb_pct,away_trb_pct,away_ast_pct,away_stl_pct,away_blk_pct,away_tov_pct,away_off_rtg,away_def_rtg
0,atl,40.000000,88.200000,0.454600,12.700000,37.000000,0.336400,22.400000,27.300000,0.816400,...,0.266200,23.690000,76.170000,49.590000,57.580000,7.510000,10.330000,11.520000,111.850000,114.450000
1,bos,42.000000,87.600000,0.481400,11.700000,29.500000,0.398700,15.800000,20.800000,0.754900,...,0.280300,20.590000,75.780000,48.290000,58.730000,8.270000,9.120000,12.880000,112.540000,113.350000
2,brk,45.222222,89.888889,0.502667,15.555556,38.222222,0.413000,19.111111,23.000000,0.832333,...,0.243778,26.555556,78.688889,50.800000,58.655556,7.055556,8.311111,9.622222,118.755556,122.100000
3,chi,44.300000,89.000000,0.498800,13.600000,35.200000,0.379300,16.400000,21.100000,0.793400,...,0.296100,21.350000,80.440000,49.160000,58.290000,8.410000,7.370000,12.260000,114.280000,114.570000
4,cho,42.150000,93.125000,0.452900,19.225000,40.125000,0.477000,20.750000,25.150000,0.818225,...,0.269500,24.465000,74.680000,51.877500,67.372500,7.942500,12.077500,13.107500,116.517500,121.910000
5,cle,36.800000,86.100000,0.426900,8.800000,26.700000,0.327000,14.800000,21.600000,0.681700,...,0.216200,25.700000,76.210000,51.830000,59.990000,10.160000,9.680000,14.210000,110.600000,99.290000
6,dal,40.846154,88.846154,0.459231,12.230769,37.153846,0.328462,17.076923,22.000000,0.770308,...,0.277385,23.407692,81.538462,52.207692,54.076923,6.838462,8.446154,11.246154,112.161538,112.861538
7,den,43.900000,88.400000,0.498400,13.900000,34.900000,0.394900,15.200000,19.400000,0.792000,...,0.212800,22.360000,74.340000,47.960000,65.390000,7.980000,6.650000,12.730000,112.210000,119.430000
8,det,40.333333,90.555556,0.444889,12.444444,37.111111,0.334444,17.000000,21.444444,0.770889,...,0.282222,22.744444,77.566667,52.433333,64.466667,6.333333,11.044444,14.122222,119.444444,110.788889
9,gsw,39.125000,92.250000,0.429000,12.500000,37.625000,0.335375,17.125000,22.625000,0.748625,...,0.313125,25.575000,84.937500,56.737500,62.725000,7.025000,8.887500,14.187500,114.175000,102.187500
